In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Reshape, Input, LSTM, Conv1D, MaxPooling1D, Flatten

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import numpy as np

In [7]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

X_train = Train.drop('STATUS', axis=1)
y_train = Train['STATUS']

X_test = Test.drop('STATUS', axis=1)
y_test = Test['STATUS']

In [8]:
def train_dnn_with_grid_search(X, y, n_splits=5, layer_configs=None):
    if layer_configs is None:
        layer_configs = [
            [64, 32, 16],  # Example architecture 1
            [128, 64],     # Example architecture 2
            [32, 16, 8],   # Example architecture 3
            [256, 128, 64] # Example architecture 4
        ]
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    
    best_config = None
    best_score = -np.inf
    results = []

    for config in layer_configs:
        print(f"Testing configuration: {config}")
        
        accuracies = []
        recalls = []
        precisions = []
        f1_scores = []
        kappas = []

        for train_index, test_index in kf.split(X):
            X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
            y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

            # Define the model for the current architecture
            model = Sequential([Input(shape=(X.shape[1],))])
            for units in config:
                model.add(Dense(units, activation='relu'))
            model.add(Dense(1, activation='sigmoid'))  # Output layer

            model.compile(optimizer=Adam(), loss='binary_crossentropy')

            # Early stopping
            early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
            model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=32, verbose=0, callbacks=[early_stopping])

            y_pred = (model.predict(X_test_fold, verbose=0) > 0.5).astype("int32")
            
            accuracies.append(accuracy_score(y_test_fold, y_pred))
            recalls.append(recall_score(y_test_fold, y_pred))
            precisions.append(precision_score(y_test_fold, y_pred))
            f1_scores.append(f1_score(y_test_fold, y_pred))
            kappas.append(cohen_kappa_score(y_test_fold, y_pred))
        
        avg_accuracy = np.mean(accuracies)
        avg_f1_score = np.mean(f1_scores)

        results.append({
            "config": config,
            "accuracy": avg_accuracy,
            "f1_score": avg_f1_score,
            "precision": np.mean(precisions),
            "recall": np.mean(recalls),
            "kappa": np.mean(kappas),
            "accuracy_std": np.std(accuracies),
            "f1_std": np.std(f1_scores)
        })

        if avg_f1_score > best_score:
            best_score = avg_f1_score
            best_config = config

    # Print results for all configurations
    print("\nGrid Search Results:")
    for result in results:
        print(f"Config: {result['config']}, "
              f"Accuracy: {result['accuracy']:.3f} ± {result['accuracy_std']:.3f}, "
              f"F1-Score: {result['f1_score']:.3f} ± {result['f1_std']:.3f}, "
              f"Precision: {result['precision']:.3f}, "
              f"Recall: {result['recall']:.3f}, "
              f"Kappa: {result['kappa']:.3f}")

    print(f"\nBest Configuration: {best_config} with F1-Score: {best_score:.3f}")


In [9]:
def train_mlp(hidden_layer_sizes, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []
    kappas = []
    
    for train_index, test_index in kf.split(X):
        X_train2, X_test2 = X.iloc[train_index], X.iloc[test_index]
        y_train2, y_test2 = y.iloc[train_index], y.iloc[test_index]

        model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=1, max_iter=5000)
        model.fit(X_train2, y_train2)

        y_pred = model.predict(X_test2)

        accuracies.append(accuracy_score(y_test2, y_pred))
        recalls.append(recall_score(y_test2, y_pred, average='weighted'))
        precisions.append(precision_score(y_test2, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test2, y_pred, average='weighted'))
        kappas.append(cohen_kappa_score(y_test2, y_pred))


    print(f'MLP Architecture: {hidden_layer_sizes}')
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}\n")


In [10]:
train_dnn_with_grid_search(X_train, y_train)

Architectures = [[5], [10], [5, 5], [5, 10], [10, 10]]
for arch in Architectures:
    train_mlp(arch, X_train, y_train)


Testing configuration: [64, 32, 16]
Testing configuration: [128, 64]
Testing configuration: [32, 16, 8]
Testing configuration: [256, 128, 64]

Grid Search Results:
Config: [64, 32, 16], Accuracy: 0.950 ± 0.004, F1-Score: 0.950 ± 0.004, Precision: 0.947, Recall: 0.954, Kappa: 0.901
Config: [128, 64], Accuracy: 0.955 ± 0.002, F1-Score: 0.955 ± 0.003, Precision: 0.959, Recall: 0.950, Kappa: 0.911
Config: [32, 16, 8], Accuracy: 0.949 ± 0.003, F1-Score: 0.949 ± 0.003, Precision: 0.951, Recall: 0.947, Kappa: 0.899
Config: [256, 128, 64], Accuracy: 0.963 ± 0.004, F1-Score: 0.963 ± 0.004, Precision: 0.960, Recall: 0.966, Kappa: 0.926

Best Configuration: [256, 128, 64] with F1-Score: 0.963
MLP Architecture: [5]
Average Accuracy: 0.916 ± 0.011
Average Recall: 0.916 ± 0.011
Average Precision: 0.916 ± 0.011
Average F1-Score: 0.916 ± 0.011
Average Kappa: 0.831 ± 0.023

MLP Architecture: [10]
Average Accuracy: 0.935 ± 0.004
Average Recall: 0.935 ± 0.004
Average Precision: 0.935 ± 0.004
Average F1-S